In [9]:
from copy import copy
from dataclasses import dataclass
from typing import Any, List


def evaluate(a):
    try:
        return copy(a.eval())
    except AttributeError:
        return copy(a)


def validate(a):
    if not (a in [0, 1] or isinstance(a, (str, And, Or, Xor, Not))):
        raise ValueError(
            "Value must be 0, 1, or an instance of str, And, Or, Xor, or Not."
        )
    else:
        return copy(a)


@dataclass
class And:
    a: Any
    b: Any

    def __init__(self, a, b):
        self.a = validate(a)
        self.b = validate(b)

    def eval(self):
        match evaluate(self.a), evaluate(self.b):
            case (0, _) | (_, 0):
                return 0
            case (Not(a), b) | (a, Not(b)) if a == b:
                return 0
            case (1, a) | (a, 1):
                return a
            case _:
                return self

    def __str__(self):
        r = self.eval()
        if r in [0, 1] or isinstance(r, str):
            return f"{r}"
        else:
            return f"({self.a}&{self.b})"


@dataclass
class Or:
    a: Any
    b: Any

    def __init__(self, a, b):
        self.a = validate(a)
        self.b = validate(b)

    def eval(self):
        match evaluate(self.a), evaluate(self.b):
            case (1, _) | (_, 1):
                return 1
            case (0, a) | (a, 0):
                return a
            case (Not(a), b) | (a, Not(b)) if a == b:
                return 1
            case _:
                return self

    def __str__(self):
        r = self.eval()
        if r in [0, 1] or isinstance(r, str):
            return f"{r}"
        else:
            return f"({self.a}|{self.b})"


@dataclass
class Xor:
    a: Any
    b: Any

    def __init__(self, a, b):
        self.a = validate(a)
        self.b = validate(b)

    def eval(self):
        match evaluate(self.a), evaluate(self.b):
            case (0, 1) | (1, 0):
                return 1
            case (0, 0) | (1, 1):
                return 0
            case (0, a) | (a, 0):
                return a
            case (1, a) | (a, 1):
                return Not(a)
            case (a, b) if a == b:
                return 0
            case (Not(a), b) | (a, Not(b)) if a == b:
                return 1
            case (Xor(a, b), c) | (a, Xor(b, c)) if a == b:
                return c
            case (Xor(a, b), c) | (a, Xor(b, c)) if a == c:
                return b
            case (Xor(a, b), c) | (a, Xor(b, c)) if b == c:
                return a
            case _:
                return copy(self)

    def __str__(self):
        return f"({self.a}^{self.b})"


@dataclass
class Not:
    a: Any

    def __init__(self, a):
        self.a = validate(a)

    def eval(self):
        match evaluate(self.a):
            case 0:
                return 1
            case 1:
                return 0
            case Not(a):
                return a
            case _:
                return copy(self)

    def __str__(self):
        r = self.eval()
        if r in [0, 1]:
            return f"{r}"
        return f"!{self.a}"

In [10]:
# Example usage:
assert And("a", 0).eval() == 0
assert And("a", 1).eval() == "a"
assert And("a", Not("a")).eval() == 0
assert Or("a", 1).eval() == 1
assert Or("a", 0).eval() == "a"
assert Or("a", Not("a")).eval() == 1
assert Xor("a", 1).eval() == Not("a")
assert Xor("a", 0).eval() == "a"
assert Xor("a", Not("a")).eval() == 1
assert Xor("a", "a").eval() == 0
assert Xor("a", Xor("a", "b")).eval() == "b"
assert Not(Not("a")).eval() == "a"

In [15]:
@dataclass
class BitArray:
    bits: List

    def len(self):
        """Returns the length of the BitArray"""
        return len(self.bits)

    def __and__(self, other):
        """Bitwise AND"""
        if self.len() != other.len():
            raise ValueError("BitArray lengths must be equal.")
        return BitArray([And(a, b).eval() for a, b in zip(self.bits, other.bits)])

    def __or__(self, other):
        """Bitwise OR"""
        if self.len() != other.len():
            raise ValueError("BitArray lengths must be equal.")
        return BitArray([Or(a, b).eval() for a, b in zip(self.bits, other.bits)])

    def __xor__(self, other):
        """Bitwise XOR"""
        if self.len() != other.len():
            raise ValueError("BitArray lengths must be equal.")
        return BitArray([Xor(a, b).eval() for a, b in zip(self.bits, other.bits)])

    def __invert__(self):
        """Bitwise NOT"""
        return BitArray([Not(a).eval() for a in self.bits])

    def __iand__(self, other):
        """In-place bitwise AND"""
        return self & other

    def __ior__(self, other):
        """In-place bitwise OR"""
        return self | other

    def __ixor__(self, other):
        """In-place bitwise XOR"""
        return self ^ other

    def __lshift__(self, n):
        """Shift left by n bits"""
        zeros = [0] * (self.len() - n)
        return BitArray(self.bits[n:] + zeros)

    def __rshift__(self, n):
        """Shift right by n bits"""
        zeros = [0] * n
        return BitArray(zeros + self.bits[:n])


In [ ]:
# Example usage:
a = BitArray(["a", "b", "c", "d"])
b = BitArray([1, 1, 0, 0])

assert (a & b) == BitArray(["a", "b", 0, 0])
assert (a | b) == BitArray([1, 1, "c", "d"])
assert (a ^ b) == BitArray([Not("a"), Not("b"), "c", "d"])
assert (~a) == BitArray([Not("a"), Not("b"), Not("c"), Not("d")])
assert (a << 2) == BitArray(["c", "d", 0, 0])
assert (a >> 2) == BitArray([0, 0, "a", "b"])